<h1><center> Assignment 1: EDA United Nations General Debate Corpus  </center></h1>



## Read UNGDC

In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import pickle
import nltk
import seaborn as sns

from nltk.sentiment import SentimentIntensityAnalyzer
import matplotlib.pyplot as plt
#nltk.download('punkt')
#nltk.download('stopwords')
nltk.download('vader_lexicon')


sns.set_style('darkgrid') # darkgrid, white grid, dark, white and ticks
plt.rc('axes', titlesize=18)     # fontsize of the axes title
plt.rc('axes', labelsize=14)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=13)    # fontsize of the tick labels
plt.rc('ytick', labelsize=13)    # fontsize of the tick labels
plt.rc('legend', fontsize=13)    # legend fontsize
plt.rc('font', size=13)          # controls default text sizes


sessions = np.arange(25, 78)
data=[]

for session in sessions:
    directory = "./TXT/Session "+str(session)+" - "+str(1945+session)
    for filename in os.listdir(directory):
        f = open(os.path.join(directory, filename), encoding = "utf8")
        if filename[0]==".": #ignore hidden files
            continue
        splt = filename.split("_")
        data.append([session, 1945+session, splt[0], f.read()])

        
df_speech = pd.DataFrame(data, columns=['Session','Year','ISO-alpha3 Code','Speech'])
df_codes = pd.read_csv('UNSD — Methodology.csv', sep=';')

df_un_merged = df_speech.merge(df_codes[['Country or Area','ISO-alpha3 Code', 'Region Name']], how='left', left_on='ISO-alpha3 Code', right_on='ISO-alpha3 Code')
df_un_merged.set_index(['Year','ISO-alpha3 Code'], inplace=True)
df_un_merged["Speech"] = df_un_merged["Speech"].str.lower()




[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/wolfson/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


## Load/Create Sentiment Intensity Analyzer and merge it with UN data


In [2]:
sia = SentimentIntensityAnalyzer()

try:
    with open('sia.pickle', 'rb') as handle:
        sia_columns = pickle.load(handle)
except:
    x=df_un_merged["Speech"].apply(sia.polarity_scores)
    sia_columns = pd.DataFrame(x.to_list())
    with open('sia.pickle', 'wb') as handle:
        pickle.dump(sia_colomns, handle)

df_un_merged = df_un_merged.reset_index().merge(sia_columns, how='left', left_index=True, right_index=True)
df_un_merged.set_index(['Year','ISO-alpha3 Code'], inplace=True)

In [3]:
display(df_un_merged)

Session  \
Year ISO-alpha3 Code            
1970 LBR                   25   
     TTO                   25   
     KWT                   25   
     SOM                   25   
     SDN                   25   
...                       ...   
2020 PRY                   75   
     NAM                   75   
2021 UKR                   76   
2022 RUS                   77   
     UKR                   77   

                                                                 Speech  \
Year ISO-alpha3 Code                                                      
1970 LBR              49.\t it gives me great pleasure, mr. presiden...   
     TTO              135.\t  before commencing my statement, i shou...   
     KWT              1.\t   mr. president, your election to the aug...   
     SOM              107.\t it is with great pleasure that my deleg...   
     SDN              126.\t in this anniversary year the general as...   
...                                                                 ...   
2020 PRY              president of the general assembly,\nexcellenci...   
     NAM              your excellency, volkan bozkir, president of t...   
2021 UKR              only pronounced in the un general assembly in ...   
2022 RUS              seventy-seventh session,\n \n12th & 13th meeti...   
     UKR              statement\nby president of ukraine h.e. mr. vo...   

                          Country or Area Region Name    neg    neu    pos  \
Year ISO-alpha3 Code                                                         
1970 LBR                          Liberia      Africa  0.080  0.719  0.201   
     TTO              Trinidad and Tobago    Americas  0.040  0.816  0.144   
     KWT                           Kuwait        Asia  0.082  0.760  0.158   
     SOM                          Somalia      Africa  0.105  0.731  0.163   
     SDN                            Sudan      Africa  0.113  0.730  0.157   
...                                   ...         ...    ...    ...    ...   
2020 PRY                         Paraguay    Americas  0.098  0.725  0.177   
     NAM                          Namibia      Africa  0.067  0.721  0.213   
2021 UKR                          Ukraine      Europe  0.084  0.779  0.137   
2022 RUS               Russian Federation      Europe  0.128  0.736  0.136   
     UKR                          Ukraine      Europe  0.167  0.661  0.172   

                      compound  
Year ISO-alpha3 Code            
1970 LBR                1.0000  
     TTO                0.9999  
     KWT                0.9997  
     SOM                0.9999  
     SDN                0.9997  
...                        ...  
2020 PRY                0.9964  
     NAM                0.9997  
2021 UKR                0.9993  
2022 RUS                0.9417  
     UKR               -0.9366  

[8484 rows x 8 columns]

## Run bag of words

In [ ]:
count_vect = CountVectorizer(lowercase=True, \
                             stop_words='english',\
                             analyzer='word',\
                            token_pattern=r"[a-z]+'?[a-z]+", \
                            min_df=100, ngram_range=(1,1))

count_vect_bigram = CountVectorizer(lowercase=True, \
                             stop_words='english',\
                             analyzer='word',\
                            token_pattern=r"[a-z]+'?[a-z]+", \
                            min_df=100, ngram_range=(2,2))

X_counts = count_vect.fit_transform(df_un_merged["Speech"])
X_counts_bigram = count_vect_bigram.fit_transform(df_un_merged["Speech"])

X_counts_tf = TfidfTransformer(use_idf=True).fit_transform(X_counts)
words_list = count_vect.get_feature_names_out()
words_list_bigram = count_vect_bigram.get_feature_names_out()
X_counts = pd.DataFrame(X_counts.toarray(), columns=words_list)
X_counts_bigram = pd.DataFrame(X_counts_bigram.toarray(), columns=words_list_bigram)
#X_counts = pd.DataFrame(X_counts_tf.toarray(), columns=words_list)

# EDA

In [ ]:
amount_of_words = X_counts.sum().sort_values(ascending=False)
amount_of_words_bigram = X_counts_bigram.sum().sort_values(ascending=False)

In [ ]:
cloud = WordCloud(scale = 10, random_state = 1, max_font_size=100, background_color="white", colormap="inferno")\
.generate_from_frequencies(amount_of_words)

plt.figure(figsize=(10,10))
plt.imshow(cloud, interpolation='bilinear')
plt.axis('off')
plt.show()

In [ ]:
cloud = WordCloud(scale = 10, random_state = 1, max_font_size=100, background_color="white", colormap="inferno")\
.generate_from_frequencies(amount_of_words_bigram)

plt.figure(figsize=(10,10))
plt.imshow(cloud, interpolation='bilinear')
plt.axis('off')
plt.show()

In [ ]:
plt.figure(figsize=(5,5))

colors = sns.color_palette('inferno')

amount_of_words[0:10].plot.bar(color = colors)
plt.xticks(rotation=30, ha = 'right')
plt.xlabel("Word")
plt.ylabel("Count")
plt.title("Top 10 most used words")

plt.figure(figsize=(5,5))

amount_of_words_bigram[0:10].plot.bar(color = colors)
plt.xticks(rotation=30, ha = 'right')
plt.xlabel("Word")
plt.ylabel("Count")
plt.title("Top 10 most used combination of words")
plt.show()

## Merge UNGDC with bag of words

In [ ]:
df_un_merged = df_un_merged.reset_index().merge(X_counts, how='left', left_index=True, right_index=True)
df_un_merged.set_index(['Year','ISO-alpha3 Code'], inplace=True)


## Read happiness report and merge with country codes to get ISO-alpha3 Code

In [ ]:
happinessdataframe = pd.read_excel('DataForTable2.1.xls')
df_happiness_merged = happinessdataframe.merge(df_codes[['Country or Area','ISO-alpha3 Code']], how='left', left_on='Country name', right_on='Country or Area')
df_happiness_merged.set_index(['year','ISO-alpha3 Code'], inplace=True)
df_happiness_merged.index.rename(['Year','ISO-alpha3 Code'], inplace=True)
df_happiness_merged.drop(columns=['Country name','Country or Area'], inplace=True)

## Read HDI report

In [ ]:
HDI = pd.read_csv('HDI.csv')
HDI.sort_values(by=['Year'], inplace = True)
HDI.set_index(['Year', 'Code'], inplace = True)
HDI.drop(columns=['Entity'], inplace=True)
HDI.index.rename(['Year','ISO-alpha3 Code'], inplace=True)

## Merge UNGDC with happiness report and HDI by Multiple index Year, ISO

In [ ]:
df_merged = df_un_merged.merge(df_happiness_merged, how='left', left_index=True, right_index=True)
df_merged = df_merged.merge(HDI, how='left', left_index=True, right_index=True)


## Select only 2005-2020

In [ ]:
df_merged_05_20 = df_merged.loc[2005:2020,:] 
df_merged_05_20


In [ ]:
#Group dataset by region
eur = df_merged_05_20[df_merged_05_20['Region Name'] == 'Europe']
afr = df_merged_05_20[df_merged_05_20['Region Name'] == 'Africa']
ame = df_merged_05_20[df_merged_05_20['Region Name'] == 'Americas']
asi = df_merged_05_20[df_merged_05_20['Region Name'] == 'Asia']
oce = df_merged_05_20[df_merged_05_20['Region Name'] == 'Oceania']

print('nr European countries:', eur['Country or Area'].nunique())
print('nr African countries:', afr['Country or Area'].nunique())
print('nr American countries:', ame['Country or Area'].nunique())
print('nr Asian countries:', asi['Country or Area'].nunique())
print('nr Oceanean countries:', oce['Country or Area'].nunique())

#Group by means per year
eur_means = eur.groupby('Year').aggregate(np.nanmean)
afr_means = afr.groupby('Year').aggregate(np.nanmean)
ame_means = ame.groupby('Year').aggregate(np.nanmean)
asi_means = asi.groupby('Year').aggregate(np.nanmean)
oce_means = oce.groupby('Year').aggregate(np.nanmean)

#Get the count of the usage of the word 'climate' in speeches
eur_climate = eur[['climate']].groupby('Year').mean()
afr_climate = afr[['climate']].groupby('Year').mean()
ame_climate = ame[['climate']].groupby('Year').mean()
asi_climate = asi[['climate']].groupby('Year').mean()
oce_climate = oce[['climate']].groupby('Year').mean()



## Create speeches countplot per region from 2005 to 2020

In [ ]:
#Plot per region (absolute counts)
plt.plot(eur_climate, label = "Europe")
plt.plot(afr_climate, label = "Africa")
plt.plot(asi_climate, label = "Asia")
plt.plot(ame_climate, label = "Americas")
plt.plot(oce_climate, label = "Oceania")
plt.legend(loc=2, prop={'size': 12})
plt.title(label = 'Mean count of the word climate in UN speeches from 2005 to 2020')
plt.xlabel('Year')
plt.ylabel('Mean count')

# #Plot per region (relative counts to nr of countries per region)
# plt.plot(eur_climate / eur['Country or Area'].nunique(), label = "Europe")
# plt.plot(afr_climate / afr['Country or Area'].nunique(), label = "Africa")
# plt.plot(asi_climate / asi['Country or Area'].nunique(), label = "Asia")
# plt.plot(ame_climate / ame['Country or Area'].nunique(), label = "Americas")
# plt.plot(oce_climate / oce['Country or Area'].nunique(), label = "Oceania")
# plt.legend(loc=2, prop={'size': 12})
# plt.title(label = 'Mean count of the word climate in UN speeches from 2005 to 2020')
# plt.xlabel('Year')
# plt.ylabel('Mean count')

In [ ]:
plt.figure(figsize=(12,5))
plt.scatter(eur['climate'], eur['Life Ladder'], label = "Europe")
# plt.scatter(afr['climate'], afr['Life Ladder'], label = "Africa")
# plt.scatter(asi['climate'], asi['Life Ladder'], label = "Asia")
# plt.scatter(ame['climate'], ame['Life Ladder'], label = "Americas")
# plt.scatter(oce['climate'], oce['Life Ladder'], label = "Oceania")
plt.legend(loc=4, prop={'size': 10})
plt.xlabel('climate mean')
plt.ylabel('Life Ladder mean')
plt.figure(figsize=(12,5))
plt.scatter(eur_climate['climate'], eur_means['Human Development Index (UNDP)'], label = "Europe")
plt.scatter(afr_climate['climate'], afr_means['Human Development Index (UNDP)'], label = "Africa")
plt.scatter(asi_climate['climate'], asi_means['Human Development Index (UNDP)'], label = "Asia")
plt.scatter(ame_climate['climate'], ame_means['Human Development Index (UNDP)'], label = "Americas")
plt.scatter(oce_climate['climate'], oce_means['Human Development Index (UNDP)'], label = "Oceania")
plt.legend(loc=4, prop={'size': 10})
plt.xlabel('climate mean')
plt.ylabel('Human Development Index')

In [ ]:
lst1 = [eur.shape[0], afr.shape[0], asi.shape[0], ame.shape[0], oce.shape[0]]
lst2 = [eur['Country or Area'].nunique(), afr['Country or Area'].nunique(), asi['Country or Area'].nunique(), ame['Country or Area'].nunique(), oce['Country or Area'].nunique()]

speeches_count_regions = pd.DataFrame(list(zip(lst1, lst2)), columns = ['Number of speeches', 'Number of countries'], index = ['Europe', 'Africa', 'Asia', 'Americas', 'Oceania'])
speeches_count_regions

## Create plots for Life ladder, GDP and HDI

In [ ]:
#Plots of happiness indicators
plt.figure(1)
plt.plot(eur_means['Life Ladder'], label = "Europe")
plt.plot(afr_means['Life Ladder'], label = "Africa")
plt.plot(asi_means['Life Ladder'], label = "Asia")
plt.plot(ame_means['Life Ladder'], label = "Americas")
plt.plot(oce_means['Life Ladder'], label = "Oceania")
plt.legend(loc=2, prop={'size': 10})
plt.title(label = 'Life ladder')
plt.xlabel('Year')
plt.ylabel('Life ladder')
plt.xticks(np.arange(2005, 2020, step=2))

# plt.plot(eur_means['Log GDP per capita'], label = "Europe")
# plt.plot(afr_means['Log GDP per capita'], label = "Africa")
# plt.plot(asi_means['Log GDP per capita'], label = "Asia")
# plt.plot(ame_means['Log GDP per capita'], label = "Americas")
# plt.plot(oce_means['Log GDP per capita'], label = "Oceania")
# plt.legend(loc=2, prop={'size': 10})
# plt.title(label = 'Log GDP')
# plt.xlabel('Year')
# plt.ylabel('Log GDP')
# plt.xticks(np.arange(2005, 2020, step=2))
plt.figure(2)

plt.plot(eur_means['Human Development Index (UNDP)'], label = "Europe")
plt.plot(afr_means['Human Development Index (UNDP)'], label = "Africa")
plt.plot(asi_means['Human Development Index (UNDP)'], label = "Asia")
plt.plot(ame_means['Human Development Index (UNDP)'], label = "Americas")
plt.plot(oce_means['Human Development Index (UNDP)'], label = "Oceania")
plt.legend(loc=2, prop={'size': 10})
plt.title(label = 'HDI')
plt.xlabel('Year')
plt.ylabel('HDI')
plt.xticks(np.arange(2005, 2020, step=2))


### Split data set

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline

# remove na from rows
x=df_merged_05_20.dropna(subset=['Life Ladder'])
y = x['Life Ladder'].values

features=['pos','neg']
features.extend(amount_of_words[2:20].keys())





# splitting the data
x_rem, x_test, y_rem, y_test = train_test_split(x, y, test_size=0.3, random_state = 42)
x_train, x_valid, y_train, y_valid = train_test_split(x_rem, y_rem, test_size=0.3, random_state = 42)






### Hyperparameters tunning based grid search on validation data set

In [ ]:
# Create grid of parameters to test through cross-validation
param_grid = {'polynomialfeatures__degree': np.arange(1,5)}

pipe = make_pipeline( PolynomialFeatures(), LinearRegression())
grid = GridSearchCV(pipe, param_grid, cv=5)
grid.fit(x_valid[features], y_valid);


### Fit the best estimator on train set

In [ ]:
grid.best_estimator_.fit(x_train[features], y_train)

In [ ]:
y_pred = grid.best_estimator_.predict(x_test[features])


# Compute test error and variance score
print("Model accuracy:")
print("Mean squared error: %.2f" % mean_squared_error(y_test, y_pred))
print('Variance score: %.2f' % r2_score(y_test, y_pred))
print("")

## Random forest and gradient boost

In [ ]:
# Create grid of parameters to test through cross-validation
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
param_grid = {'max_depth': [10,20, 30, 50, 70]}
grid_forest = GridSearchCV(RandomForestRegressor(random_state=0), param_grid, cv=5)
grid_forest.fit(x_valid[features], y_valid);

In [ ]:
y_pred = grid_forest.best_estimator_.predict(x_test[features])
print(grid_forest.best_estimator_)
# Compute test error and variance score
print("Model accuracy:")
print("Mean squared error: %.2f" % mean_squared_error(y_test, y_pred))
print('Variance score: %.2f' % r2_score(y_test, y_pred))
print("")

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
reg = GradientBoostingRegressor(random_state=0)
reg.fit(x_train[features], y_train)
y_pred = reg.predict(x_test[features])

# Compute test error and variance score
print("Model accuracy:")
print("Mean squared error: %.2f" % mean_squared_error(y_test, y_pred))
print('Variance score: %.2f' % r2_score(y_test, y_pred))
print("")
